In [15]:
import os
import string
import numpy as np
import tweepy                                                                       # pip install tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import gspread
#from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from tqdm import tqdm
import time
import re
import preprocessor as p                                                             # pip install tweet-preprocessor                                                    
from nltk.tokenize import word_tokenize
from textblob import TextBlob                                                        # pip install textblob 
from nltk.corpus import stopwords
import pygsheets
from tkinter import *
import pycountry
from tkinter import messagebox
from tkinter import ttk
import webbrowser 
from dotenv import load_dotenv
load_dotenv('john.env')

True

In [2]:
consumer_key = os.getenv('TWITTER_API')
consumer_secret = os.getenv('TWITTER_SECRET_KEY')

In [3]:
type(consumer_key)

str

In [4]:
callback_uri = 'oob'                                                      # declare a uniform resource indicator, and set to 'out-of-band'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_uri)   # creates an authentication object. stores our consumer and secret key. 
redirect_url = auth.get_authorization_url()                               # stores the API authorization url used to get a PIN number                             
webbrowser.open(redirect_url)                                             # Opens web and redirects you to Twitter app authorization website. Click Authorize and it provides a PIN number.
prompt = '>'                                                              # Enter the PIN number here. 
print('enter PIN number at the prompt. Hit enter.')
user_PIN = input(prompt)                                                  # enter the PIN number in prompt and now you'll have an access token. 
auth.get_access_token(user_PIN)                                           # Uses the PIN to generate an access token.
print('Your access key ', auth.access_token)                              # This key does not change regardless how many different PIN numbers you get.
print('The secret key ', auth.access_token_secret)
api = tweepy.API(auth)                                                    # Create a tweepy API object with the access token. Set wait_on_rate_limit to reset after 15 minutes.

enter PIN number at the prompt. Hit enter.
Your access key  18325825-cHvbx5Kpg6HFjIrFYo8wfr8jheadKCXjUNQG57BvU
The secret key  Bk14J5TH5g4YmZvRmra7u23uwrkfwgWrNHHs0v4G0Ttby


In [9]:
api.update_status(status='Test')

Status(_api=<tweepy.api.API object at 0x000002189A404BC8>, _json={'created_at': 'Sun Aug 01 23:31:06 +0000 2021', 'id': 1421976817099952129, 'id_str': '1421976817099952129', 'text': 'Test', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">FINTECH_CLASS_UW</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 18325825, 'id_str': '18325825', 'name': 'jb_bees', 'screen_name': 'JB_Bees', 'location': 'PA', 'description': 'none of it has been my fault so far.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 17, 'friends_count': 25, 'listed_count': 1, 'created_at': 'Tue Dec 23 03:57:12 +0000 2008', 'favourites_count': 229, 'utc_offset': None, 'time_zone': None, 'geo_en

In [5]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
   ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [13]:
def clean_tweets(tweet):

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)

    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'[~\x00-\x7F]+', '', tweet)


    tweet = emoji_pattern.sub(r'', tweet)

    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []

    for w in word_tokens:

        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)

    return ' '.join(filtered_tweet)

In [7]:
#Start of GUI
window = Tk()
#Title of Gui
window.title("Sentiment Analysis")
#multiple tabs
tab_control = ttk.Notebook(window)

tab1 = ttk.Frame(tab_control)

tab2 = ttk.Frame(tab_control)

tab_control.add(tab1, text='System')

tab_control.add(tab2, text='About')
#GUI window size
window.geometry('1000x700')
heading = Label(tab1, text="Welcome To Sentiment Analysis of tweets", font=("Arial Bold", 25))
#GUI fields position
heading.grid(column=1, row=0)
lbl = Label(tab1, text="Please enter your keyword", font=("Times New Roman", 20))
lbl.grid(column=1, row=10)
txt = Entry(tab1,width=10)

txt.grid(column=2, row=10)
prgbar = Label(tab1, text="Current status: Awaiting input from user", font=("Times New Roman", 10))                        # progress bar visual
prgbar.grid(column=3,row =12)

#tab2
abt = Label(tab2,text="Welcome To Sentiment Analysis of tweets", font=("Arial Bold", 25))
abt.grid(column=1, row=0)
abt2 = Label(tab2,text="Good Day", font=("Times New Roman", 15))
abt2.grid(column=1, row=1)
abt3 = Label(tab2,text="This program takes an input of a keyword from user and extracts latest 200-250 tweets relating to the keyword.", font=("Times New Roman", 15))
abt3.grid(column=1, row=2)
abt4 = Label(tab2,text="After extraction these tweets are analyzed exploratory and sentimentally before being presented visually", font=("Times New Roman", 15))
abt4.grid(column=1, row=3)
abt5 = Label(tab2,text="For more information please contact: Name", font=("Times New Roman", 15))
abt5.grid(column=1, row=4)
abt6 = Label(tab2,text="Email: ", font=("Times New Roman", 10))
abt6.grid(column=1, row=5)



#process once the button is clicked
def clicked():
    messagebox.showinfo('Message', 'The analysis has begun')

    prgbar.configure(text="Current status: extracting tweets")

In [11]:
#extract keyword from user
search_words = 'dogecoin'

# Collect tweets
tweets = tweepy.Cursor(
    api.search,
    q=search_words,
    lang="en"
    ).items(500)

#print extraction of tweets
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)
    print(f"new tweets retrieved: {len(tweets_copy)}")
    prgbar.configure(text=f"Current status: Tweets extracted, {len(tweets_copy)}" )

1it [00:00,  3.15it/s]

new tweets retrieved: 1
new tweets retrieved: 2
new tweets retrieved: 3
new tweets retrieved: 4
new tweets retrieved: 5
new tweets retrieved: 6
new tweets retrieved: 7
new tweets retrieved: 8
new tweets retrieved: 9
new tweets retrieved: 10
new tweets retrieved: 11
new tweets retrieved: 12
new tweets retrieved: 13


14it [00:00, 21.08it/s]

new tweets retrieved: 14
new tweets retrieved: 15
new tweets retrieved: 16
new tweets retrieved: 17
new tweets retrieved: 18
new tweets retrieved: 19
new tweets retrieved: 20
new tweets retrieved: 21
new tweets retrieved: 22
new tweets retrieved: 23
new tweets retrieved: 24
new tweets retrieved: 25
new tweets retrieved: 26
new tweets retrieved: 27


28it [00:01, 26.15it/s]

...

new tweets retrieved: 463
new tweets retrieved: 464
new tweets retrieved: 465
new tweets retrieved: 466
new tweets retrieved: 467
new tweets retrieved: 468
new tweets retrieved: 469
new tweets retrieved: 470
new tweets retrieved: 471
new tweets retrieved: 472
new tweets retrieved: 473
new tweets retrieved: 474
new tweets retrieved: 475
new tweets retrieved: 476
new tweets retrieved: 477


478it [00:11, 46.37it/s]

new tweets retrieved: 478
new tweets retrieved: 479
new tweets retrieved: 480
new tweets retrieved: 481
new tweets retrieved: 482
new tweets retrieved: 483
new tweets retrieved: 484
new tweets retrieved: 485
new tweets retrieved: 486
new tweets retrieved: 487
new tweets retrieved: 488
new tweets retrieved: 489
new tweets retrieved: 490
new tweets retrieved: 491


500it [00:12, 40.67it/s]

new tweets retrieved: 492
new tweets retrieved: 493
new tweets retrieved: 494
new tweets retrieved: 495
new tweets retrieved: 496
new tweets retrieved: 497
new tweets retrieved: 498
new tweets retrieved: 499
new tweets retrieved: 500


In [16]:
#creation of dataframe
polarity = []
subjectivity = []
Sentiment = []
filtered_tweet = []
clean_text = []
country = []
fltr_1 = []
index = []
Sentiment_rating = []

tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    clean_text = p.clean(tweet.text)
    filtered_tweet = clean_tweets(clean_text)
    blob = TextBlob(filtered_tweet)
    Sentiment_rating = blob.sentiment
    polarity = Sentiment_rating.polarity
    #splitting 1st word
    #fltr_1 = filtered_tweet.split()[0]
    #sentiment in text output
    subjectivity = Sentiment_rating.subjectivity
    if polarity >= 0.1 and polarity <= 0.3:
        Sentiment = " slightly positive"
    elif polarity < -0.1 and polarity > -0.3:
        Sentiment = "slightly negative"
    elif polarity >= 0.3:
        Sentiment = "positive"
    elif polarity < -0.3:
        Sentiment = "negative"
    else:
        Sentiment = "neutral"
        #country name (where applicable)
    for country in pycountry.countries:
        if country.name in tweet.user.location:
            country = country.name
        else:
            country = "Not Available"
            #hashtags (where applicalble)
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'tweet_id': tweet.id,
                                           'tweet_location': country,
                                           'polarity': polarity,
                                           'subjectivity': subjectivity,
                                           'Sentiment': Sentiment,
                                           'date': tweet.created_at,
                                           'text': tweet.text,
                                           #'filtered_tweet': fltr_1,
                                            'filtered_tweet': filtered_tweet,
                                           'hashtags': [hashtags if hashtags else None],
                                           'source': tweet.source,
                                           'likes': tweet.favorite_count,
                                           'is_retweet': tweet.retweeted}, index=[0]))
    tweets_df.head()

  0%|          | 0/500 [00:00<?, ?it/s]


TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

In [17]:
#EXPORTING DATAFRAME ONTO GOOGLESHEETS
client= pygsheets.authorize(service_file='filepath')
lbl.configure(text="Authorized")

sheet = client.open("fyp_tw")
prgbar.configure(text="Current status: Sheet opened")

wks = sheet[0]
prgbar.configure(text="Current status: First sheet Accessed")

wks.set_dataframe(tweets_df,(1,1))
prgbar.configure(text="Current status: Data Updated")

FileNotFoundError: [Errno 2] No such file or directory: 'filepath'

In [ ]:
webbrowser.open("your public tableau link")

btn = Button(tab1, text="Click Me to start Analysis", command=clicked)

btn.grid(column=3, row=10)
tab_control.pack(expand=1, fill='both')

window.mainloop()
